In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from faicons import icon_svg
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
from shinywidgets import render_plotly, output_widget, render_widget
from shiny.express import input, render, ui
from ipyleaflet import Map as IpylMap, GeoJSON as IpylGeoJSON, Popup
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Popup


In [3]:

# === Load population data ===
# Expect columns: "Alter", "Männer", "Frauen", and optionally "Stadtteil"
df_pyr = pd.read_excel("C:/Users/24634/Desktop/Dashboard/Dashboard/Input/2022.xlsx")
df_st = gpd.read_file("C:/Users/24634/Desktop/Dashboard/Dashboard/Input/stadtteil.geojson")
bv = pd.read_csv("C:/Users/24634/Desktop/Dashboard/Dashboard/Input/bevoelkerung.csv")
# Debug: Print available columns in the GeoDataFrame
#print("Available columns in df_st:", df_st.columns.tolist())
#print("Sample properties:", df_st.iloc[0].to_dict() if len(df_st) > 0 else "No data")


In [9]:
# Debug: Print available columns in the GeoDataFrame
print("Available columns in df_st:", df_st.columns.tolist())
if len(df_st) > 0:
    print("Sample properties:", df_st.iloc[0].to_dict())

Available columns in df_st: ['PGIS_ID', 'MIFSTADTT_', 'MIFSTADTT0', 'MIFSTADTT1', 'MIFSTADTT2', 'MIFSTADTT3', 'MIFSTADTT4', 'MIFSTADTT5', 'MIFSTADTT6', 'MIFSTADTT7', 'MIFSTADTT8', 'MIFSTADTT9', 'MIFSTADT10', 'MIFSTADT11', 'MIFSTADT12', 'geometry']
Sample properties: {'PGIS_ID': '{CB32ECD6-BDA6-49AA-977F-DBD60DE9EE2E}', 'MIFSTADTT_': '21.06.2021', 'MIFSTADTT0': '17.02.2015', 'MIFSTADTT1': 25776, 'MIFSTADTT2': None, 'MIFSTADTT3': 1126.44, 'MIFSTADTT4': '31', 'MIFSTADTT5': '{9670693A-B0EC-4D60-854F-F9DCA2C89D52}', 'MIFSTADTT6': 'Oggersheim', 'MIFSTADTT7': '{CB32ECD6-BDA6-49AA-977F-DBD60DE9EE2E}', 'MIFSTADTT8': '{9670693A-B0EC-4D60-854F-F9DCA2C89D52}', 'MIFSTADTT9': 0, 'MIFSTADT10': '15:47:00', 'MIFSTADT11': '12:18:00', 'MIFSTADT12': '{00000000-0000-0000-0000-000000000000}', 'geometry': <MULTIPOLYGON Z (((8.346 49.49 0, 8.346 49.49 0, 8.347 49.49 0, 8.348 49.491...>}


In [3]:
df_st = gpd.read_file("C:/Users/24634/Desktop/Dashboard/Dashboard/Input/stadtteil.geojson")

In [12]:


# Mittelpunkt für Karte berechnen
map_center = [49.48, 8.44]

# Erstelle Folium-Karte mit CartoDB positron (Carto Light)
m = folium.Map(location=map_center, zoom_start=12, tiles='CartoDB positron')

# Stadtteile als GeoJson hinzufügen
GeoJson(
    df_st.__geo_interface__,
    name='Stadtteile',
    style_function=lambda x: {
        'fillColor': 'gray',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.1
    }
).add_to(m)

In [13]:
m

In [ ]:

df_st = gpd.read_file("C:/Users/24634/Desktop/Dashboard/Dashboard/Input/stadtteil.geojson")
#bv = pd.read_csv("../Input/bevoelkerung.csv")
df_st_local = df_st
# Ensure WGS84 for leaflet
df_st = df_st.set_crs(epsg=25832).to_crs(epsg=4326)


geojson_data = df_st_local.__geo_interface__
map_center = [49.48, 8.44]

m = Map(center=map_center, zoom=9, close_popup_on_click=False)

marker = Marker(location=[49.48, 8.44])
m.add(marker)

message1 = HTML()
message2 = HTML()
# message1.value = "Try clicking the marker!"
message2.value = "Hello <b>World</b>"
message2.placeholder = "Some HTML"
message2.description = "Some HTML"

# Create GeoJSON layer with popup enabled
geo_layer = IpylGeoJSON(
    data=geojson_data,
    style={
        "color": "red",
        "weight": 2,
        "fillColor": "lightblue",
        "fillOpacity": 0.5,
    },
    hover_style={
        "fillColor": "orange",
        "fillOpacity": 0.7,
        "weight": 3,
    },
    # Enable popups - this will automatically show popups when features are clicked
    popup_property='popup'
)

# Popup associated to a layer
marker.popup = message2

m

In [13]:
from ipyleaflet import Map, Marker, GeoJSON
from ipywidgets import HTML
import geopandas as gpd

# Load and prepare the data
df_st = gpd.read_file("C:/Users/24634/Desktop/Dashboard/Dashboard/Input/stadtteil.geojson")
# Ensure WGS84 for leaflet
df_st = df_st.set_crs(epsg=25832).to_crs(epsg=4326)

# Prepare popup content for each feature
for idx, row in df_st.iterrows():
    properties = row.to_dict()
    name = properties.get("MIFSTADTT6", "Unbekannter Stadtteil")
    einwohner = properties.get("MIFSTADTT1", "k.A.")
    flaeche = properties.get("MIFSTADTT3", "k.A.")

    popup_content = f"""
    <div style='padding: 10px; min-width: 200px;'>
        <h4 style='margin: 0 0 10px 0; color: #2c3e50;'>{name}</h4>
        <div style='border-top: 1px solid #eee; padding-top: 8px;'>
            <p style='margin: 5px 0;'><b>Einwohner:</b> {einwohner}</p>
            <p style='margin: 5px 0;'><b>Fläche:</b> {flaeche} ha</p>
        </div>
    </div>
    """
    df_st.at[idx, 'popup'] = popup_content

geojson_data = df_st.__geo_interface__
map_center = [49.48, 8.44]

# Create map
m = Map(center=map_center, zoom=11)

# Add marker at center
marker = Marker(location=[49.48, 8.44])
m.add(marker)

# Create popup content for marker
message2 = HTML()
message2.value = "Hello <b>World</b><br>This is Ludwigshafen!"
marker.popup = message2

# Create GeoJSON layer
geo_layer = GeoJSON(
    data=geojson_data,
    style={
        "color": "red",
        "weight": 2,
        "fillColor": "lightblue",
        "fillOpacity": 0.5,
    },
    hover_style={
        "fillColor": "orange",
        "fillOpacity": 0.7,
        "weight": 3,
    },
    popup_property='popup'
)

m.add(geo_layer)
m

RuntimeError: shinywidgets requires that all ipywidgets be constructed within an active Shiny session